In [32]:
import os
import fitz
from tqdm import tqdm

In [33]:
def check_rate_function(rate_1, rate_2, possible_rate):
        # 檢查兩種方法提取的評價是否相同
    if rate_1 == rate_2 and rate_1!= None:
        return rate_1
    elif rate_1 in possible_rate:
        return rate_1
    elif rate_2 in possible_rate:
        return rate_2
    else :
        return 'NULL'
            

In [45]:
zip_path = '國泰'
count_true = 0
possible_rate = ['中立 – 維持中立', '買進 – 維持買進', '買進', '中立 – 初次評等中立', '中立 – 買進轉中立', '中立 – 初次評等', '買進– 維持買進']
for directory in tqdm(os.listdir(zip_path)):
    directory_path = os.path.join(zip_path, directory)
    rate_1 = None
    rate_2 = None
    rate = None
    try:
        with fitz.open(directory_path) as doc:
            mat = fitz.Matrix(1, 1)
            page = doc.load_page(0)
            rect = page.rect
            page_check_source = doc.load_page(-1)
            text_check_source = page_check_source.get_text()
            if '國泰金融控股公司' in text_check_source :
                # 檢查版本                
                clip_check_report = fitz.Rect(rect.width/2, 0, rect.width, 150)
                text_check_report = page.get_text(clip=clip_check_report)
                if any(keyword in text_check_report for keyword in ['個股報告']):
                    # 提取評價的第一種方法
                    try:
                        clip_old_report_1 = fitz.Rect(350, 80, rect.width, 160)
                        text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
                        rate_1 = text_old_report_1.split('\n')[0].strip()
                    except:
                        rate_1 = None
                        # 提取評價的第二種方法
                    try:
                        clip_old_report_2 = fitz.Rect(350, 80, 550, 130)
                        rate_2 = page.get_text(clip=clip_old_report_2).strip()
                    except:
                        rate_2 = None
                # else :
                #     print(f'{text_check_report}, {directory_path} NULL')
                rate = check_rate_function(rate_1, rate_2, possible_rate)
            else:
                # False
                print(f'{directory_path} is not posted by 國泰投顧')
    except:
        print(directory_path, 'fail')
    if rate == 'NULL' :
        print('case1', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
    elif rate == 'wrong':
        count_true += 1
        # print(directory_path, rate,
        #       f'text_check_new_report={text_check_new_report}, text_check_old_fast_report={text_check_old_fast_report}')
    elif rate != None :
        count_true += 1
        # print(directory_path, rate,
        #         f'rate_1={rate_1}, rate_2={rate_2}')
    else :
        print('case2', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
# 計算成功和失敗的數量並打印
count_false = len(os.listdir(zip_path)) - count_true
success_rate = round((count_true/len(os.listdir(zip_path))), 3)*100
print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')

 75%|███████▍  | 53/71 [00:03<00:01, 16.73it/s]

國泰\6409_旭隼_大和國泰_Hold.pdf is not posted by 國泰投顧
case2 國泰\6409_旭隼_大和國泰_Hold.pdf None rate_1=None, rate_2=None


100%|██████████| 71/71 [00:04<00:00, 14.51it/s]

共71件, 成功70件, 失敗1件, 成功率:98.6%


In [46]:
import fitz
doc = fitz.open(r'國泰\6409_旭隼_大和國泰_Hold.pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [47]:
# 檢查是否由第一金投顧出版
page_check_source = doc.load_page(-1)
text_check_source = page_check_source.get_text()
if '國泰金融控股公司' in text_check_source :
    print('True')

In [48]:
# 檢查版本
clip_check_report = fitz.Rect(rect.width/2, 0, rect.width, 100)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_report)
pix.save("page-clip_check_report.png")
text_check_report = page.get_text(clip=clip_check_report)
if '個股報告' in text_check_report :
    print('個股報告')
else :
    print('NULL')
# text_check_report

NULL


In [49]:
# 報告為新版個股報告 提取評價的第一種方法
try:
    clip_old_report_1 = fitz.Rect(350, 80, rect.width, 160)
    text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
    pix.save("page-clip_new_report_1.png")
    # text_old_report_1 = text_old_report_1.split('投資評等')[1].strip()
    rate_1 = text_old_report_1.split('\n')[0].strip()
except:
    rate_1 = None
rate_1

''

In [50]:
# 報告為新版個股報告 提取評價的第二種方法
try:
    clip_old_report_2 = fitz.Rect(350, 80, 550, 130)
    rate_2 = page.get_text(clip=clip_old_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_2)
    pix.save("page-clip_new_report_2.png")
except:
    rate_2 = None
rate_2

''